# 03. 특허 원문 수집 (KIPRIS PDF + Google Patents)

이 노트북은 RAG(Retrieval-Augmented Generation) 벡터스토어 구축을 위해 특허의 원문(Full-text) 데이터를 수집하고 전처리합니다.

## 주요 기능
1. **타겟 특허 (한국) 수집**:
   - **KIPRIS Plus API** (`getPubFullTextInfoSearch`)를 사용하여 공개전문 PDF URL을 획득합니다.
   - PDF를 다운로드하고 `pypdf` 라이브러리를 사용하여 텍스트를 추출합니다.
   - 추출된 텍스트에서 주요 섹션(초록, 명세서, 청구범위)을 파싱합니다.

2. **선행기술 (한국 및 해외) 수집**:
   - **한국 특허**: KIPRIS PDF 방식을 우선 시도하고, 실패 시 Google Patents 크롤링으로 대체(Fallback)합니다.
   - **해외 특허**: Google Patents를 크롤링하여 원문을 확보합니다.
   - **다국어 처리**: 한국어, 영어, 중국어, 일본어 등 다국어 텍스트의 인코딩(UTF-8)을 보장합니다.

3. **데이터 저장**:
   - 수집된 텍스트는 `data/processed/fulltext/` 디렉토리에 구조화된 텍스트 파일로 저장됩니다.
   - 메타데이터는 JSON 파일로 저장되어 추후 추적 가능합니다.

## 사전 요구사항
- `KIPRIS_API_KEY`가 `.env` 파일에 설정되어 있어야 합니다.
- 필요한 라이브러리: `requests`, `beautifulsoup4`, `xmltodict`, `pypdf`, `tqdm`

In [1]:
import os
import re
import time
import json
import random
import threading
from pathlib import Path
from typing import Dict, List, Any, Optional
from io import BytesIO

import requests
import xmltodict
from bs4 import BeautifulSoup, UnicodeDammit
from dotenv import load_dotenv
from tqdm.auto import tqdm
from pypdf import PdfReader

# 환경 설정
def find_repo_root(start: Path | None = None) -> Path:
    cur = (start or Path.cwd()).resolve()
    for p in (cur, *cur.parents):
        if (p / "pyproject.toml").exists() and (p / "data").exists():
            return p
    return cur

REPO_ROOT = find_repo_root()
load_dotenv(REPO_ROOT / "env")
load_dotenv(REPO_ROOT / ".env")

KIPRIS_API_KEY = os.getenv("KIPRIS_API_KEY", "")
if not KIPRIS_API_KEY:
    raise ValueError("KIPRIS_API_KEY가 설정되지 않았습니다. env 파일을 확인하세요.")

# KIPRIS API 설정
BASE_URL = "http://plus.kipris.or.kr/kipo-api/kipi/patUtiModInfoSearchSevice"
PATH_PUB_FULLTEXT = "getPubFullTextInfoSearch"  # 공개전문 PDF
PATH_ANN_FULLTEXT = "getAnnFullTextInfoSearch"  # 공고전문 PDF (공개 실패 시 fallback, applicationNumber 사용)
PATH_REG_FULLTEXT = "getRegFullTextInfoSearch"  # 등록전문 PDF (추가 fallback; 스펙/파라미터는 상황에 따라 다를 수 있음)

# 출력 디렉토리
OUT_DIR = REPO_ROOT / "data" / "processed" / "fulltext"
PDF_DIR = REPO_ROOT / "data" / "processed" / "fulltext_pdfs"
OUT_DIR.mkdir(parents=True, exist_ok=True)
PDF_DIR.mkdir(parents=True, exist_ok=True)

# 논문 정합성: examiner-only GT(=paper mode)로 구축한 데이터셋을 우선 사용
_dataset_candidates = [
    REPO_ROOT / "data" / "processed" / "kipris_semiconductor_ai_dataset_paper.jsonl",
    REPO_ROOT / "data" / "processed" / "kipris_semiconductor_ai_dataset_paper.checkpoint.jsonl",
    REPO_ROOT / "data" / "processed" / "kipris_semiconductor_ai_dataset.jsonl",
    REPO_ROOT / "data" / "processed" / "kipris_semiconductor_ai_dataset.checkpoint.jsonl",
 ]
DATASET_PATH = next((p for p in _dataset_candidates if p.exists()), _dataset_candidates[0])
if not DATASET_PATH.exists():
    raise FileNotFoundError(
        "데이터셋 파일을 찾지 못했습니다. 먼저 notebooks/02_kipris_dataset_builder.ipynb를 MODE='paper'로 실행해\n"
        "data/processed/kipris_semiconductor_ai_dataset_paper.jsonl (또는 checkpoint) 를 생성하세요."
    )

print(f"✓ Repo Root: {REPO_ROOT}")
print(f"✓ API Key: {KIPRIS_API_KEY[:4]}...")
print(f"✓ Dataset: {DATASET_PATH}")
print(f"✓ Output Dir: {OUT_DIR}")
print(f"✓ PDF Cache: {PDF_DIR}")

# 수집 설정
SAMPLE_SIZE = 100  # 타겟 특허 수집 개수(거절-only 모집단에서 앞에서부터 N개)
COHORT = "reject_only"  # "reject_only" | "all"
SAMPLING_METHOD = "head"  # "head"(순차 수집) | "random"(실험용)
SAMPLING_SEED = 42  # random 샘플 재현성
MIN_REQUEST_INTERVAL = 0.5  # KIPRIS 요청 간격(초)
GOOGLE_PATENTS_DELAY = 2.0  # Google Patents 요청 간격(초)
MAX_WORKERS = 1  # 본 노트북은 기본값으로 '순차 수집'합니다. (병렬은 rate limit 위험)

✓ Repo Root: /home/arkwith/Dev/paper_data
✓ API Key: aggT...
✓ Dataset: /home/arkwith/Dev/paper_data/data/processed/kipris_semiconductor_ai_dataset_paper.jsonl
✓ Output Dir: /home/arkwith/Dev/paper_data/data/processed/fulltext
✓ PDF Cache: /home/arkwith/Dev/paper_data/data/processed/fulltext_pdfs


In [2]:
# KIPRIS PDF API 함수 (공개전문→공고전문→(옵션)등록전문 fallback + 실패 사유 수집)

_kipris_lock = threading.Lock()
_kipris_last_request = 0.0

class KiprisServiceKeyError(RuntimeError):
    pass

def _throttle_kipris():
    """KIPRIS API rate limiting"""
    global _kipris_last_request
    with _kipris_lock:
        elapsed = time.monotonic() - _kipris_last_request
        if elapsed < MIN_REQUEST_INTERVAL:
            time.sleep(MIN_REQUEST_INTERVAL - elapsed)
        _kipris_last_request = time.monotonic()

def kipris_call(path: str, params: Dict[str, Any], max_retries: int = 3) -> tuple[Dict[str, Any], Dict[str, Any]]:
    """KIPRIS API 호출 후 (header, body) 반환.

    - 실패하더라도 header/resultCode/resultMsg를 최대한 보존해서 반환
    - '서비스키/권한 만료'만 치명적 예외로 처리 (그 외는 실패 사유로 기록하고 계속 진행)
    """
    url = f"{BASE_URL}/{path}"
    params = params.copy()
    params["ServiceKey"] = KIPRIS_API_KEY

    last_exc: Optional[Exception] = None
    for attempt in range(max_retries):
        try:
            _throttle_kipris()
            resp = requests.get(url, params=params, timeout=30)
            resp.raise_for_status()
            data = xmltodict.parse(resp.text)
            response = data.get("response", {}) or {}
            header = response.get("header", {}) or {}
            body = response.get("body", {}) or {}

            if str(header.get("successYN", "")).strip().upper() == "N":
                msg = str(header.get("resultMsg", "") or "").strip()
                # 서비스키/권한 만료만 즉시 중단
                if any(x in msg for x in ["서비스 이용 권한", "서비스키", "만료", "잘못", "SERVICEKEY", "SERVICE KEY"]):
                    raise KiprisServiceKeyError(msg or "KIPRIS ServiceKey error")
            return header, body
        except KiprisServiceKeyError:
            raise
        except Exception as e:
            last_exc = e
            if attempt == max_retries - 1:
                return {"successYN": "N", "resultCode": "HTTP/CLIENT", "resultMsg": str(e)}, {}
            time.sleep(1 * (attempt + 1))

    return {"successYN": "N", "resultCode": "UNKNOWN", "resultMsg": str(last_exc) if last_exc else "unknown"}, {}

def _extract_pdf_url_from_body(body: Dict[str, Any]) -> Optional[str]:
    item = (body.get("item", {}) or {}) if isinstance(body, dict) else {}
    if isinstance(item, list):
        item = item[0] if item else {}
    pdf_url = ""
    if isinstance(item, dict):
        pdf_url = str(item.get("path", "") or "")
    return pdf_url if (pdf_url and pdf_url.startswith("http")) else None

def get_kipris_pdf_url_with_reason(*, identifier: str, operation: str, id_param: str) -> tuple[Optional[str], Dict[str, Any]]:
    """KIPRIS에서 PDF URL을 얻고, 실패 사유(header 등)를 함께 반환."""
    header, body = kipris_call(operation, {id_param: identifier})
    pdf_url = _extract_pdf_url_from_body(body)
    info = {
        "operation": operation,
        "id_param": id_param,
        "identifier": identifier,
        "successYN": str(header.get("successYN", "")).strip(),
        "resultCode": str(header.get("resultCode", "")).strip(),
        "resultMsg": str(header.get("resultMsg", "")).strip(),
        "has_body": bool(body),
        "has_pdf_url": bool(pdf_url),
    }
    return pdf_url, info

def extract_text_from_pdf_bytes(pdf_bytes: bytes) -> Optional[str]:
    try:
        reader = PdfReader(BytesIO(pdf_bytes))
        text_parts = []
        for page in reader.pages:
            text = page.extract_text()
            if text:
                text_parts.append(text)
        full_text = "\n".join(text_parts)
        return full_text if full_text.strip() else None
    except Exception:
        return None

def download_and_extract_pdf(pdf_url: str, cache_path: Optional[Path] = None) -> Optional[str]:
    """PDF 다운로드 및 텍스트 추출 (cache_path가 있으면 재사용)"""
    try:
        if cache_path and cache_path.exists():
            return extract_text_from_pdf_bytes(cache_path.read_bytes())

        resp = requests.get(pdf_url, timeout=60)
        resp.raise_for_status()
        if cache_path:
            cache_path.write_bytes(resp.content)
        return extract_text_from_pdf_bytes(resp.content)
    except Exception as e:
        print(f"Failed to download/extract PDF from {pdf_url[:50]}...: {e}")
        return None

def parse_patent_sections(text: str, app_num: str) -> Dict[str, str]:
    """특허 텍스트를 섹션별로 파싱 (간단 휴리스틱)"""
    result = {
        "application_number": app_num,
        "full_text": text,
        "abstract": "",
        "description": "",
        "claims": ""
    }

    for keyword in ["【요약】", "【발명의 요약】", "[요약]"]:
        if keyword in text:
            parts = text.split(keyword)
            if len(parts) > 1:
                content = parts[1]
                if "【" in content:
                    content = content.split("【")[0]
                result["abstract"] = content.strip()[:2000]
                break

    for keyword in ["【청구범위】", "[청구범위]", "【특허청구범위】"]:
        if keyword in text:
            parts = text.split(keyword)
            if len(parts) > 1:
                content = parts[1]
                if "【" in content:
                    content = content.split("【")[0]
                result["claims"] = content.strip()
                break

    for keyword in ["【발명의 상세한 설명】", "[발명의 상세한 설명]"]:
        if keyword in text:
            parts = text.split(keyword)
            if len(parts) > 1:
                content = parts[1]
                if "【" in content:
                    content = content.split("【")[0]
                result["description"] = content.strip()
                break

    return result

def _canonical_pdf_cache_path(cache_stem: str) -> Path:
    """PDF 캐시 파일명은 '특허번호만.pdf' 형태로 통일."""
    return PDF_DIR / f"{cache_stem}.pdf"

def _pick_cache_path(cache_stem: str, operation: str) -> Optional[Path]:
    """기존 캐시 호환:
    1) <stem>.pdf (신규 표준)
    2) <stem>.<operation>.pdf (과거 생성 파일)
    """
    p_std = _canonical_pdf_cache_path(cache_stem)
    if p_std.exists():
        return p_std
    p_old = PDF_DIR / f"{cache_stem}.{operation}.pdf"
    if p_old.exists():
        return p_old
    return None

def _attempt_operation(*, clean_id: str, operation: str, id_param: str, cache_stem: str, application_number: str, tried: list[dict]) -> Optional[Dict[str, str]]:
    # 0) 이미 PDF 캐시가 있으면 API 호출 없이 재추출
    cache_path = _pick_cache_path(cache_stem, operation)
    if cache_path is not None:
        text = extract_text_from_pdf_bytes(cache_path.read_bytes())
        tried.append({"operation": operation, "resultCode": "CACHE", "resultMsg": "used_cached_pdf", "cache_path": str(cache_path)})
        if text:
            result = parse_patent_sections(text, application_number)
            result["source"] = "kipris_pdf"
            result["kipris_operation"] = operation
            result["pdf_url"] = str(cache_path)
            return result
        tried.append({"operation": operation, "resultCode": "PDF", "resultMsg": "cached_pdf_extract_failed", "cache_path": str(cache_path)})
        # 캐시가 깨졌을 수도 있으니 계속해서 API 시도
    else:
        tried.append({"operation": operation, "resultCode": "CACHE", "resultMsg": "no_cached_pdf"})

    # 1) API로 PDF URL 획득 → 다운로드/캐시/추출
    pdf_url, info = get_kipris_pdf_url_with_reason(identifier=clean_id, operation=operation, id_param=id_param)
    tried.append(info)
    if not pdf_url:
        return None

    # 신규 표준 파일명: <특허번호>.pdf
    cache_path = _canonical_pdf_cache_path(cache_stem)
    text = download_and_extract_pdf(pdf_url, cache_path)
    if not text:
        tried.append({"operation": operation, "resultCode": "PDF", "resultMsg": "download_or_extract_failed"})
        return None

    result = parse_patent_sections(text, application_number)
    result["source"] = "kipris_pdf"
    result["kipris_operation"] = operation
    result["pdf_url"] = pdf_url
    return result

def get_kipris_fulltext_with_reason(application_number: str, *, registration_number: Optional[str] = None) -> tuple[Optional[Dict[str, str]], Dict[str, Any]]:
    """KIPRIS PDF 파이프라인:
    공개전문(applicationNumber) → 공고전문(applicationNumber) → (옵션)등록전문(registrationNumber) fallback + 실패 사유 반환.

    최적화: PDF 캐시가 있으면 API 호출 없이 재추출합니다.
    """
    clean_app_num = re.sub(r"[^0-9]", "", str(application_number))
    clean_reg_num = re.sub(r"[^0-9]", "", str(registration_number)) if registration_number else ""
    cache_stem = re.sub(r"[^a-zA-Z0-9_-]", "_", clean_app_num)
    tried: list[dict] = []
    used_operation: Optional[str] = None

    # 1) 공개전문: applicationNumber
    result = _attempt_operation(
        clean_id=clean_app_num,
        operation=PATH_PUB_FULLTEXT,
        id_param="applicationNumber",
        cache_stem=cache_stem,
        application_number=application_number,
        tried=tried,
    )
    if result:
        used_operation = PATH_PUB_FULLTEXT
        return result, {"ok": True, "tried": tried, "used_operation": used_operation}

    # 2) 공고전문: applicationNumber
    result = _attempt_operation(
        clean_id=clean_app_num,
        operation=PATH_ANN_FULLTEXT,
        id_param="applicationNumber",
        cache_stem=cache_stem,
        application_number=application_number,
        tried=tried,
    )
    if result:
        used_operation = PATH_ANN_FULLTEXT
        return result, {"ok": True, "tried": tried, "used_operation": used_operation}

    # 3) (옵션) 등록전문: registrationNumber (없으면 스킵)
    if clean_reg_num:
        result = _attempt_operation(
            clean_id=clean_reg_num,
            operation=PATH_REG_FULLTEXT,
            id_param="registrationNumber",
            cache_stem=cache_stem,
            application_number=application_number,
            tried=tried,
        )
        if result:
            used_operation = PATH_REG_FULLTEXT
            return result, {"ok": True, "tried": tried, "used_operation": used_operation}
    else:
        tried.append({"operation": PATH_REG_FULLTEXT, "resultCode": "SKIP", "resultMsg": "no_registration_number"})

    return None, {"ok": False, "tried": tried, "used_operation": used_operation}

# 기존 코드 호환용 (기존 호출부가 있으면 동작)
def get_kipris_fulltext(application_number: str) -> Optional[Dict[str, str]]:
    result, _ = get_kipris_fulltext_with_reason(application_number)
    return result

print("✓ KIPRIS PDF 함수 정의 완료 (PDF 캐시 파일명: <특허번호>.pdf)")

✓ KIPRIS PDF 함수 정의 완료 (PDF 캐시 파일명: <특허번호>.pdf)


In [3]:
# Google Patents 크롤링 함수 (인코딩/정규화/비특허(NPL) 필터 보강 + 섹션/문단 보존 강화)

_google_last_request = 0.0
_google_lock = threading.Lock()

# 특허 문서번호처럼 보이는지(간단 휴리스틱)
_PATENT_PREFIXES = (
    "US", "KR", "JP", "CN", "EP", "WO", "DE", "FR", "GB", "TW",
    "CA", "AU", "RU", "IN", "BR", "IT", "ES"
)


def _throttle_google():
    """Google Patents rate limiting"""
    global _google_last_request
    with _google_lock:
        elapsed = time.monotonic() - _google_last_request
        if elapsed < GOOGLE_PATENTS_DELAY:
            time.sleep(GOOGLE_PATENTS_DELAY - elapsed)
        _google_last_request = time.monotonic()


def looks_like_patent_doc_number(value: str) -> bool:
    """선행기술 문자열이 특허 문서번호 형태인지 판별.

    - 논문/문헌(NPL)은 Google Patents로 가져올 수 없어 404가 다량 발생하므로 미리 제외
    - 완벽한 규칙이 아니라 '오류 감소' 목적의 휴리스틱
    """
    if not value:
        return False

    s = str(value).strip()

    # 아주 흔한 NPL 패턴
    if any(x in s for x in ["arXiv", "Vol.", "pp.", "doi", "et al", "IEEE", "ACM", "Journal", "Proceedings"]):
        return False

    # JP평성... 같은 비정형 라벨 제외
    if re.search(r"[가-힣]", s) and not s.upper().startswith("KR"):
        return False

    up = s.upper().replace(" ", "")
    if not up.startswith(_PATENT_PREFIXES):
        return False

    # 숫자가 충분히 있어야 함
    digits = re.findall(r"\d+", up)
    if not digits:
        return False
    if len("".join(digits)) < 6:
        return False

    return True


def normalize_patent_number(doc_num: str) -> str:
    """문서번호를 Google Patents URL 형식으로 정규화

    목표:
    - 공백/슬래시/하이픈 제거
    - 'KR10...' → 'KR...' 변환
    - 'KR102430483 B1' 같이 중간 공백이 있는 kind code도 결합
    """
    s = str(doc_num).strip()

    # 슬래시/공백/하이픈 제거
    normalized = re.sub(r"[\s/\-]", "", s)

    # 괄호/따옴표 등 URL에 불필요한 문자 제거 (영숫자만 남김)
    normalized = re.sub(r"[^0-9A-Za-z]", "", normalized)

    # KR10 → KR (한국 특허 정규화)
    normalized = re.sub(r"^KR10", "KR", normalized, flags=re.IGNORECASE)

    return normalized


def is_korean_patent(doc_num: str) -> bool:
    """한국 특허인지 판별"""
    return str(doc_num).strip().upper().startswith("KR")


def guess_google_patents_lang(doc_num: str) -> str:
    """문서번호 접두어로 Google Patents 언어 경로를 추정.

    - JP/CN/KR은 해당 언어로 먼저 시도하면 CJK 텍스트 보존/표시가 더 안정적인 경우가 있음
    """
    up = str(doc_num).strip().upper()
    if up.startswith("JP"):
        return "ja"
    if up.startswith("CN"):
        return "zh-CN"
    if up.startswith("KR"):
        return "ko"
    return "en"


def _repair_common_mojibake(text: str) -> str:
    """자주 관측되는 'UTF-8을 latin-1로 잘못 디코딩' 패턴 복원.

    예: '；' → bytes(ef bc 9b) → latin-1 디코딩 시 'ï¼'

    주의: 항상 안전한 변환은 아니므로, 복원이 실패하면 원문 유지.
    """
    if not text:
        return text

    if any(ch in text for ch in ["ï", "ã", "â"]):
        try:
            fixed = text.encode("latin1").decode("utf-8")
            # 복원 결과가 더 길거나 CJK 비율이 증가하는 경우만 채택
            mojibake_markers = sum(text.count(x) for x in ["ï", "ã", "â"]) 
            fixed_markers = sum(fixed.count(x) for x in ["ï", "ã", "â"]) 
            if fixed_markers < mojibake_markers:
                return fixed
        except Exception:
            pass

    return text


def _decode_html_bytes(raw: bytes) -> str:
    # 1) 먼저 UTF-8로 시도 (대부분 Google Patents는 UTF-8)
    for enc in ["utf-8", "utf-8-sig"]:
        try:
            s = raw.decode(enc)
            if "<html" in s.lower() or "<!doctype" in s.lower():
                return s
        except Exception:
            pass

    # 2) UnicodeDammit에 CJK 후보 인코딩을 알려주고 디코딩
    dammit = UnicodeDammit(
        raw,
        is_html=True,
        override_encodings=[
            "utf-8",
            "gb18030", "gbk",
            "big5",
            "shift_jis", "euc-jp",
            "cp949",
        ],
    )
    if dammit.unicode_markup:
        return dammit.unicode_markup

    # 3) 최후 fallback
    return raw.decode("utf-8", errors="replace")


def _fetch_google_patents_html(url: str) -> Optional[str]:
    """URL을 가져와 HTML 문자열로 반환.

    중요:
    - CJK(한/중/일) 텍스트가 깨지는 경우가 많아, `resp.apparent_encoding`을 강제로 쓰면
      오히려 모지바케(예: ë°...)가 발생할 수 있음.
    - bytes(resp.content) 기준으로 디코딩 후, 섹션 텍스트를 후처리에서 추가 복원한다.
    """
    _throttle_google()
    headers = {
        "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120 Safari/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    }
    resp = requests.get(url, headers=headers, timeout=30)
    if resp.status_code != 200:
        return None

    return _decode_html_bytes(resp.content)


_BLOCK_TAGS = ("p", "div", "li", "tr", "h1", "h2", "h3", "h4", "h5", "h6")


def _postprocess_google_text(text: str) -> str:
    if not text:
        return text

    text = text.replace("\r\n", "\n").replace("\r", "\n")
    text = _repair_common_mojibake(text)

    # 'AbstractA ...', 'DescriptionMultitask ...' 형태로 붙어버리는 케이스 보정
    text = re.sub(r"^(Abstract)(?=\S)", r"Abstract\n", text)
    text = re.sub(r"^(Description)(?=\S)", r"Description\n", text)

    # 번역/기계추출 텍스트에서 자주 등장하는 헤딩을 줄바꿈으로 분리
    heading_words = [
        "Technical field",
        "FIELD OF THE INVENTION",
        "Background technology",
        "BACKGROUND",
        "SUMMARY",
        "The content of the invention",
        "DETAILED DESCRIPTION",
        "BRIEF DESCRIPTION OF THE DRAWINGS",
        "DESCRIPTION OF THE DRAWINGS",
    ]
    for w in heading_words:
        # 이미 줄 시작이면 유지, 단어 앞에 글자가 붙어있으면 줄바꿈 삽입
        text = re.sub(rf"(?i)(?<!\n)({re.escape(w)})\b", r"\n\1", text)

    # 과도한 공백/빈 줄 정리
    text = re.sub(r"[ \t]+\n", "\n", text)
    text = re.sub(r"\n{3,}", "\n\n", text)
    return text.strip()


def _extract_google_section_text(section_tag) -> str:
    if not section_tag:
        return ""

    # 태그를 문자열로 새로 파싱해 조작(원 soup 변경 방지)
    frag = BeautifulSoup(str(section_tag), "html.parser")

    for t in frag(["script", "style", "noscript"]):
        t.decompose()

    for br in frag.find_all("br"):
        br.replace_with("\n")

    for tag in frag.find_all(_BLOCK_TAGS):
        tag.insert_before("\n")
        tag.insert_after("\n")

    text = frag.get_text()
    return _postprocess_google_text(text)


def get_google_patents_fulltext(doc_num: str) -> Optional[Dict[str, str]]:
    """Google Patents에서 특허 원문 크롤링

    강화 포인트:
    - CJK 문서에서 인코딩 모지바케 복원 시도
    - 섹션 내부 문단/블록 경계를 최대한 보존 (chunking/RAG 성능에 중요)
    """
    if not looks_like_patent_doc_number(doc_num):
        return None

    normalized = normalize_patent_number(doc_num)
    lang = guess_google_patents_lang(doc_num)

    # URL 후보: (1) 언어 경로, (2) /en, (3) 기본
    url_candidates = [
        f"https://patents.google.com/patent/{normalized}/{lang}",
        f"https://patents.google.com/patent/{normalized}/en",
        f"https://patents.google.com/patent/{normalized}",
    ]

    try:
        html = None
        used_url = None
        for u in url_candidates:
            html = _fetch_google_patents_html(u)
            if html:
                used_url = u
                break

        if not html:
            # 특허로 보이는 값만 제한적으로 로그
            print(f"Failed to fetch {doc_num}: HTTP error / not found")
            return None

        soup = BeautifulSoup(html, "html.parser")

        # 제목
        title_tag = soup.find("meta", {"name": "DC.title"})
        title = title_tag["content"] if title_tag and title_tag.get("content") else ""
        title = _postprocess_google_text(title)

        # Abstract / Description / Claims
        abstract_section = soup.find("section", {"itemprop": "abstract"})
        abstract = _extract_google_section_text(abstract_section)

        description_section = soup.find("section", {"itemprop": "description"})
        description = _extract_google_section_text(description_section)

        claims_section = soup.find("section", {"itemprop": "claims"})
        claims = _extract_google_section_text(claims_section)

        if not any([abstract, description, claims]):
            return None

        return {
            "doc_number": str(doc_num),
            "normalized": normalized,
            "title": title,
            "abstract": abstract,
            "description": description,
            "claims": claims,
            "source": "google_patents",
            "google_url": used_url or url_candidates[0],
            "google_lang": lang,
        }

    except Exception as e:
        print(f"Error crawling {doc_num}: {e}")
        return None


print("✓ Google Patents 크롤링 함수 정의 완료 (문단/인코딩 보존 강화)")


✓ Google Patents 크롤링 함수 정의 완료 (문단/인코딩 보존 강화)


In [4]:
# 데이터셋 로드 및 샘플링 (거절-only 기본값 + 순차 100건)

print(f"데이터셋 로드 중: {DATASET_PATH}")

dataset: List[Dict[str, Any]] = []
with open(DATASET_PATH, "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        try:
            dataset.append(json.loads(line))
        except Exception:
            continue

print(f"✓ 전체 데이터셋: {len(dataset)}건")

def _get_reg_fields(entry: Dict[str, Any]) -> tuple[str, str, Optional[bool]]:
    tp = entry.get("target_patent", {}) or {}
    reg = (((tp.get("biblio") or {}).get("registration") or {}) if isinstance(tp, dict) else {})
    status = str(reg.get("register_status") or "").strip()
    final_disposal = str(reg.get("final_disposal") or "").strip()
    is_registered = reg.get("is_registered")
    # is_registered는 bool 또는 None
    return status, final_disposal, is_registered if isinstance(is_registered, bool) else None

def _is_reject_only(entry: Dict[str, Any]) -> bool:
    # '거절-only'의 최소 정의: register_status 또는 final_disposal에 '거절'이 포함되는 경우
    status, final_disposal, _ = _get_reg_fields(entry)
    return ("거절" in status) or ("거절" in final_disposal)

# 모집단(cohort) 필터
if COHORT == "reject_only":
    cohort = [e for e in dataset if _is_reject_only(e)]
    print(f"✓ 거절-only cohort: {len(cohort)}건")
else:
    cohort = list(dataset)
    print(f"✓ cohort(all): {len(cohort)}건")

# 샘플링: 기본은 순차(head)로 100건 (재현성: app_num 정렬)
def _app_num_key(e: Dict[str, Any]) -> str:
    tp = e.get("target_patent", {}) or {}
    return str(tp.get("application_number") or "").strip()

cohort_sorted = sorted(cohort, key=_app_num_key)

if SAMPLING_METHOD == "random":
    random.seed(SAMPLING_SEED)
    sample = random.sample(cohort_sorted, min(SAMPLE_SIZE, len(cohort_sorted)))
    print(f"✓ 샘플링 완료(random, seed={SAMPLING_SEED}): {len(sample)}건")
else:
    sample = cohort_sorted[: min(SAMPLE_SIZE, len(cohort_sorted))]
    print(f"✓ 샘플링 완료(head): {len(sample)}건")

# 수집 대상 특허 번호/등록번호 추출 + 선행기술(특허/비특허) 원본 목록 확보
target_app_nums: List[str] = []
target_reg_num_by_app: Dict[str, str] = {}
prior_art_values: List[str] = []

for entry in sample:
    tp = entry.get("target_patent", {}) or {}
    app_num = str(tp.get("application_number", "")).strip()
    if not app_num:
        continue
    target_app_nums.append(app_num)

    reg_num = (((tp.get("biblio") or {}).get("registration") or {}).get("register_number"))
    if reg_num:
        target_reg_num_by_app[app_num] = re.sub(r"[^0-9]", "", str(reg_num))

    gt = entry.get("ground_truth_prior_arts") or []
    for pa in gt:
        prior_art_values.append(str(pa))

# 선행기술 중 특허 문서번호만 추출 (NPL은 별도 트랙으로 '수집(저장)' 처리)
prior_art_patent_doc_nums = sorted({v for v in prior_art_values if looks_like_patent_doc_number(v)})
prior_art_npl_values = sorted({v for v in prior_art_values if not looks_like_patent_doc_number(v)})

print(f"✓ 타겟 특허: {len(target_app_nums)}건")
print(f"  - 등록번호 보유: {len(target_reg_num_by_app)}건 (등록전문 fallback 가능)")
print(f"✓ 선행기술(원본, 중복 제거): {len(set(prior_art_values))}건")
print(f"  - 특허 문서번호로 판별됨: {len(prior_art_patent_doc_nums)}건")
print(f"  - 비특허 문헌(NPL)로 판별됨: {len(prior_art_npl_values)}건")

# 한국/해외 특허 분류(집계용)
korean_prior_arts = [doc for doc in prior_art_patent_doc_nums if is_korean_patent(doc)]
foreign_prior_arts = [doc for doc in prior_art_patent_doc_nums if not is_korean_patent(doc)]

print(f"  - 한국 특허(특허만): {len(korean_prior_arts)}건")
print(f"  - 해외 특허(특허만): {len(foreign_prior_arts)}건")

total_kipris_calls = len(target_app_nums) + len(korean_prior_arts)
print(f"\n예상 API 호출(집계 기준; 실제는 캐시/중복/순차 처리로 감소할 수 있음):")
print(f"  - KIPRIS: {total_kipris_calls}건 (약 {total_kipris_calls * MIN_REQUEST_INTERVAL / 60:.1f}분)")
print(f"  - Google Patents: {len(foreign_prior_arts)}건 (약 {len(foreign_prior_arts) * GOOGLE_PATENTS_DELAY / 60:.1f}분)")

if prior_art_npl_values:
    print(f"\n(참고) NPL 예시 3개: {prior_art_npl_values[:3]}")

# ------------------------------------------------------------
# (최적화) 이미 저장된 원문(txt)이 있으면 로드해서 API 호출을 줄임
# ------------------------------------------------------------

target_dir = OUT_DIR / "targets"
prior_art_dir = OUT_DIR / "prior_arts"
npl_dir = OUT_DIR / "npl"
target_dir.mkdir(parents=True, exist_ok=True)
prior_art_dir.mkdir(parents=True, exist_ok=True)
npl_dir.mkdir(parents=True, exist_ok=True)

def _safe_stem(value: str) -> str:
    return re.sub(r"[^a-zA-Z0-9_-]", "_", str(value))

def load_saved_patent_text_txt(path: Path) -> Optional[Dict[str, str]]:
    """save_patent_text()로 저장한 txt를 다시 dict로 복원.

    섹션 마커:
    - ## ABSTRACT / ## DESCRIPTION / ## CLAIMS / ## FULL TEXT (FROM PDF)
    """
    if not path.exists():
        return None
    try:
        raw = path.read_text(encoding="utf-8")
    except Exception:
        return None

    # 기본 메타 헤더
    doc_id = ""
    source = ""
    kipris_operation = ""
    title = ""

    lines = raw.splitlines()
    for line in lines[:30]:
        if line.startswith("Document Number:"):
            doc_id = line.split(":", 1)[1].strip()
        elif line.startswith("Source:"):
            source = line.split(":", 1)[1].strip()
        elif line.startswith("KIPRIS Operation:"):
            kipris_operation = line.split(":", 1)[1].strip()
        elif line.startswith("Title:"):
            title = line.split(":", 1)[1].strip()

    def _extract_section(marker: str) -> str:
        m = re.search(rf"^## {re.escape(marker)}\s*$", raw, flags=re.MULTILINE)
        if not m:
            return ""
        start = m.end()
        # 다음 섹션 헤더 또는 EOF
        n = re.search(r"^## ", raw[start:], flags=re.MULTILINE)
        end = start + (n.start() if n else len(raw[start:]))
        return raw[start:end].strip()

    abstract = _extract_section("ABSTRACT")
    description = _extract_section("DESCRIPTION")
    claims = _extract_section("CLAIMS")
    full_text = _extract_section("FULL TEXT (FROM PDF)")

    # 최소한 하나는 있어야 유효
    if not any([abstract, description, claims, full_text]):
        return None

    data: Dict[str, str] = {
        "title": title,
        "abstract": abstract,
        "description": description,
        "claims": claims,
        "full_text": full_text,
        "source": source or "cached_txt",
    }
    if kipris_operation:
        data["kipris_operation"] = kipris_operation
    # 타겟은 application_number, 선행기술은 doc_number로도 쓸 수 있음
    if doc_id:
        if doc_id.isdigit():
            data["application_number"] = doc_id
        else:
            data["doc_number"] = doc_id
    return data

def load_saved_npl_txt(path: Path) -> Optional[Dict[str, str]]:
    if not path.exists():
        return None
    try:
        raw = path.read_text(encoding="utf-8")
    except Exception:
        return None
    if not raw.strip():
        return None
    return {"raw": raw}

# 캐시 인덱스
cached_target_text_by_app: Dict[str, Dict[str, str]] = {}
cached_prior_art_text_by_doc: Dict[str, Dict[str, str]] = {}
cached_npl_by_key: Dict[str, Dict[str, str]] = {}

for app in target_app_nums:
    p = target_dir / f"{_safe_stem(app)}.txt"
    d = load_saved_patent_text_txt(p)
    if d:
        cached_target_text_by_app[str(app)] = d

for doc in prior_art_patent_doc_nums:
    p = prior_art_dir / f"{_safe_stem(doc)}.txt"
    d = load_saved_patent_text_txt(p)
    if d:
        cached_prior_art_text_by_doc[str(doc)] = d

# NPL 캐시는 파일명 규칙이 해시 기반이어서, 여기서는 로드를 강제하지 않고 수집 단계에서 on-demand로 처리
print("\n[캐시 상태]")
print(f"- 타겟 캐시: {len(cached_target_text_by_app)}/{len(target_app_nums)}")
print(f"- 선행기술 캐시(특허만): {len(cached_prior_art_text_by_doc)}/{len(prior_art_patent_doc_nums)}")
print(f"- NPL 후보: {len(prior_art_npl_values)} (수집 단계에서 citation/web로 저장)")

데이터셋 로드 중: /home/arkwith/Dev/paper_data/data/processed/kipris_semiconductor_ai_dataset_paper.jsonl
✓ 전체 데이터셋: 1500건
✓ 거절-only cohort: 194건
✓ 샘플링 완료(head): 100건
✓ 타겟 특허: 100건
  - 등록번호 보유: 0건 (등록전문 fallback 가능)
✓ 선행기술(원본, 중복 제거): 273건
  - 특허 문서번호로 판별됨: 254건
  - 비특허 문헌(NPL)로 판별됨: 19건
  - 한국 특허(특허만): 177건
  - 해외 특허(특허만): 77건

예상 API 호출(집계 기준; 실제는 캐시/중복/순차 처리로 감소할 수 있음):
  - KIPRIS: 277건 (약 2.3분)
  - Google Patents: 77건 (약 2.6분)

(참고) NPL 예시 3개: ['A. Sinha et al., Deep Learning 3D Shape Surfaces Using Geometry Images, ECCV 2016, Lecture Notes in Computer Science, Vol.9910(2016)', 'Aurelijus Vaitkevicius 외, Recognition of American Sign Language Gestures in a Virtual Reality Using Leap Motion 외, Applied Sciences, 2019.01.28., Vol.9, No.445, pp.1-16.', 'Comparison of glaucoma-diagnostic ability between wide-field swept-source OCT retinal nerve fiber layer maps and spectraldomain OCT, Eye (2018) 32:1483-1492']

[캐시 상태]
- 타겟 캐시: 0/100
- 선행기술 캐시(특허만): 0/254
- NPL 후보: 19 (수집 단계에서 citation/web로 저장)

In [5]:
# (진단) KIPRIS PDF URL/텍스트 추출 확인 (공개→공고→(옵션)등록 fallback + 사유 출력)
print("\n=== KIPRIS 진단: PDF URL/텍스트 추출 확인 ===")

test_apps = [
    "1020210057220",  # 이전 로그 샘플
    "1020050050026",  # 공고전문PDF(getAnnFullTextInfoSearch) 문서 스펙 샘플
 ]

for test_app in test_apps:
    print("\n---")
    print("test_app", test_app)
    try:
        result, info = get_kipris_fulltext_with_reason(test_app)
        print("ok", info.get("ok"))
        print("tried", info.get("tried"))
        if result:
            print("used_operation", result.get("kipris_operation"))
            print("extracted_chars", len(result.get("full_text", "")))
            print("text_head", (result.get("full_text", "")[:200]).replace("\n", " "))
    except KiprisServiceKeyError as e:
        print("❌ KIPRIS 서비스키/기간/권한 오류:", str(e))
        break


=== KIPRIS 진단: PDF URL/텍스트 추출 확인 ===

---
test_app 1020210057220
ok True
tried [{'operation': 'getPubFullTextInfoSearch', 'resultCode': 'CACHE', 'resultMsg': 'no_cached_pdf'}, {'operation': 'getPubFullTextInfoSearch', 'id_param': 'applicationNumber', 'identifier': '1020210057220', 'successYN': 'Y', 'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.', 'has_body': True, 'has_pdf_url': True}]
used_operation getPubFullTextInfoSearch
extracted_chars 20225
text_head (19) 대한민국특허청(KR) (12) 공개특허공보(A) (11) 공개번호   10-2022-0150031 (43) 공개일자   2022년11월10일 (51) 국제특허분류(Int. Cl.)      G01N 29/44 (2006.01)  G01N 29/06 (2006.01)      G01N 29/36 (2006.01)  G06N 3/08 (2006.01)

---
test_app 1020050050026
ok True
tried [{'operation': 'getPubFullTextInfoSearch', 'resultCode': 'CACHE', 'resultMsg': 'no_cached_pdf'}, {'operation': 'getPubFullTextInfoSearch', 'id_param': 'applicationNumber', 'identifier': '1020050050026', 'successYN': 'Y', 'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.', 'has_body': True, '

In [6]:
# 순차 수집: 타겟 특허 원문 → 해당 타겟의 GT 선행기술(특허/비특허) 원문(또는 citation/web 텍스트)
# [중요] 수집 즉시 파일로 저장하여 중단 시에도 데이터를 보존합니다.
import hashlib

print("\n=== 순차 수집 시작: (거절-only 기본) 타겟 → 선행기술(특허/비특허) ===")

# 1. 저장 경로 및 함수 정의 (수집 루프에서 즉시 사용)
target_dir = OUT_DIR / "targets"
prior_art_dir = OUT_DIR / "prior_arts"
npl_dir = OUT_DIR / "npl"
target_dir.mkdir(parents=True, exist_ok=True)
prior_art_dir.mkdir(parents=True, exist_ok=True)
npl_dir.mkdir(parents=True, exist_ok=True)

def _safe_stem(value: str) -> str:
    return re.sub(r"[^a-zA-Z0-9_-]", "_", str(value))

def save_patent_text(data: Dict[str, str], output_path: Path) -> None:
    """특허 데이터를 텍스트 파일로 저장 (UTF-8)"""
    with open(output_path, "w", encoding="utf-8", newline="\n") as f:
        doc_id = data.get("application_number") or data.get("doc_number", "")
        f.write(f"Document Number: {doc_id}\n")
        f.write(f"Source: {data.get('source', 'unknown')}\n")
        if data.get("kipris_operation"):
            f.write(f"KIPRIS Operation: {data.get('kipris_operation')}\n")
        f.write(f"Title: {data.get('title', '')}\n")
        f.write("=" * 80 + "\n\n")

        if data.get("abstract"):
            f.write("## ABSTRACT\n\n")
            f.write(data["abstract"] + "\n\n")

        if data.get("description"):
            f.write("## DESCRIPTION\n\n")
            f.write(data["description"] + "\n\n")

        if data.get("claims"):
            f.write("## CLAIMS\n\n")
            f.write(data["claims"] + "\n\n")

        if data.get("full_text") and len(data.get("full_text", "")) > 1000:
            f.write("## FULL TEXT (FROM PDF)\n\n")
            f.write(data["full_text"] + "\n\n")

def save_npl_text(rec: Dict[str, str], output_path: Path) -> None:
    """비특허(NPL) 데이터를 텍스트로 저장"""
    with open(output_path, "w", encoding="utf-8", newline="\n") as f:
        f.write(f"Source: {rec.get('source', 'citation_only')}\n")
        if rec.get("url"):
            f.write(f"URL: {rec.get('url')}\n")
        if rec.get("citation"):
            f.write(f"Citation: {rec.get('citation')}\n")
        f.write("=" * 80 + "\n\n")
        f.write(rec.get("text") or rec.get("citation") or "")

# 2. 수집 상태 변수
target_fulltext: Dict[str, Dict[str, str]] = {}
prior_art_fulltext: Dict[str, Dict[str, str]] = {}
npl_fulltext: Dict[str, Dict[str, str]] = {}

failed_targets: List[str] = []
failed_target_details: Dict[str, Dict[str, Any]] = {}
failed_prior_arts: List[str] = []
failed_npl: List[str] = []

cache_hit_target = 0
api_hit_target = 0
cache_hit_prior = 0
api_hit_kipris = 0
api_hit_google = 0
npl_cache_hit = 0
npl_web_hit = 0
npl_citation_only = 0

def _npl_key(npl: str) -> str:
    return hashlib.sha1(str(npl).encode("utf-8")).hexdigest()[:16]

def _extract_first_url(text: str) -> Optional[str]:
    if not text:
        return None
    m = re.search(r"https?://\S+", text)
    if not m:
        return None
    url = m.group(0).strip()
    url = url.rstrip(")].,;\"'”)」")
    return url if url.startswith("http") else None

def fetch_npl_text(npl: str) -> Dict[str, str]:
    raw = str(npl).strip()
    url = _extract_first_url(raw)
    if not url:
        return {"source": "citation_only", "citation": raw, "text": raw}

    try:
        _throttle_google()
        resp = requests.get(url, timeout=30, headers={"User-Agent": "Mozilla/5.0"})
        resp.raise_for_status()
        soup = BeautifulSoup(resp.text, "html.parser")
        extracted = soup.get_text("\n")
        extracted = re.sub(r"\n{3,}", "\n\n", extracted).strip()
        if len(extracted) > 300_000:
            extracted = extracted[:300_000]
        return {"source": "web", "url": url, "citation": raw, "text": extracted or raw}
    except Exception as e:
        return {"source": "citation_only", "url": url, "citation": raw, "text": raw, "error": str(e)}

def _load_cached_npl(npl: str) -> Optional[Dict[str, str]]:
    k = _npl_key(npl)
    p = npl_dir / f"{k}.txt"
    if not p.exists():
        return None
    d = load_saved_npl_txt(p)
    if not d:
        return None
    return {"source": "cached_txt", "key": k, "raw": d.get("raw", "")}

# 3. 메인 루프 (타겟 1건씩 처리 후 즉시 저장)
for entry in tqdm(sample, desc="Sequential collecting (targets→GT)"):
    tp = entry.get("target_patent", {}) or {}
    app_num = str(tp.get("application_number", "")).strip()
    if not app_num:
        continue

    # --- 1) 타겟 수집 & 저장 ---
    target_save_path = target_dir / f"{_safe_stem(app_num)}.txt"
    
    if app_num not in target_fulltext:
        cached = (globals().get("cached_target_text_by_app") or {}).get(app_num)
        if cached:
            target_fulltext[app_num] = cached
            cache_hit_target += 1
            # 캐시된 내용이라도 파일이 없으면 저장 (복구)
            if not target_save_path.exists():
                save_patent_text(cached, target_save_path)
        else:
            # 파일이 이미 존재하면 로드해서 사용 (재개)
            if target_save_path.exists():
                loaded = load_saved_patent_text_txt(target_save_path)
                if loaded:
                    target_fulltext[app_num] = loaded
                    cache_hit_target += 1
                    # (이미 파일 있으므로 저장 스킵)
                else:
                    # 파일 깨짐 등 -> 다시 수집
                    pass
            
            if app_num not in target_fulltext:
                reg_num = (target_reg_num_by_app or {}).get(str(app_num))
                try:
                    result, info = get_kipris_fulltext_with_reason(app_num, registration_number=reg_num)
                except KiprisServiceKeyError as e:
                    raise RuntimeError(f"KIPRIS 서비스키/권한 문제로 중단: {e}")
                api_hit_target += 1
                if result:
                    target_fulltext[app_num] = result
                    cached_target_text_by_app[app_num] = result
                    # [즉시 저장]
                    save_patent_text(result, target_save_path)
                else:
                    failed_targets.append(app_num)
                    failed_target_details[app_num] = info

    # --- 2) GT 선행기술 수집 & 저장 ---
    gt_list = [str(x) for x in (entry.get("ground_truth_prior_arts") or [])]
    for pa in gt_list:
        pa = str(pa).strip()
        if not pa:
            continue

        # 2-A) 특허 문서번호
        if looks_like_patent_doc_number(pa):
            doc_num = pa
            prior_save_path = prior_art_dir / f"{_safe_stem(doc_num)}.txt"

            if doc_num in prior_art_fulltext:
                continue
            
            cached = (globals().get("cached_prior_art_text_by_doc") or {}).get(doc_num)
            if cached:
                prior_art_fulltext[doc_num] = cached
                cache_hit_prior += 1
                if not prior_save_path.exists():
                    save_patent_text(cached, prior_save_path)
                continue
            
            if prior_save_path.exists():
                loaded = load_saved_patent_text_txt(prior_save_path)
                if loaded:
                    prior_art_fulltext[doc_num] = loaded
                    cache_hit_prior += 1
                    continue

            # 수집 실행
            res = None
            if is_korean_patent(doc_num):
                res = get_kipris_fulltext(doc_num)
                if res:
                    api_hit_kipris += 1
                else:
                    res = get_google_patents_fulltext(doc_num)
                    if res: api_hit_google += 1
            else:
                res = get_google_patents_fulltext(doc_num)
                if res: api_hit_google += 1
            
            if res:
                prior_art_fulltext[doc_num] = res
                cached_prior_art_text_by_doc[doc_num] = res
                # [즉시 저장]
                save_patent_text(res, prior_save_path)
            else:
                failed_prior_arts.append(doc_num)
            continue

        # 2-B) 비특허(NPL)
        npl = pa
        k = _npl_key(npl)
        npl_save_path = npl_dir / f"{k}.txt"

        if k in npl_fulltext:
            continue
        
        cached_n = _load_cached_npl(npl)
        if cached_n:
            npl_fulltext[k] = cached_n
            npl_cache_hit += 1
            continue
        
        if npl_save_path.exists():
            # 파일 있으면 로드 (단, NPL은 단순 텍스트라 다시 읽기 쉬움)
            d = load_saved_npl_txt(npl_save_path)
            if d:
                npl_fulltext[k] = {"key": k, "source": "cached_file", **d}
                npl_cache_hit += 1
                continue

        rec = fetch_npl_text(npl)
        npl_fulltext[k] = {"key": k, **rec}
        if rec.get("source") == "web":
            npl_web_hit += 1
        else:
            npl_citation_only += 1
        
        # [즉시 저장]
        save_npl_text(rec, npl_save_path)

        if rec.get("error"):
            failed_npl.append(npl)

print("\n[순차 수집 집계]")
print(f"- 타겟: {len(target_fulltext)}/{len(target_app_nums)} (cache {cache_hit_target}, api {api_hit_target})")
print(f"- 선행기술(특허): {len(prior_art_fulltext)} (cache {cache_hit_prior}, kipris {api_hit_kipris}, google {api_hit_google})")
print(f"- 선행기술(NPL): {len(npl_fulltext)} (cache {npl_cache_hit}, web {npl_web_hit}, citation_only {npl_citation_only})")
if failed_targets:
    print(f"- 타겟 실패(처음 5개): {failed_targets[:5]}")
if failed_prior_arts:
    print(f"- 선행기술(특허) 실패(처음 10개): {failed_prior_arts[:10]}")
if failed_npl:
    print(f"- NPL 웹 추출 실패(처음 3개, citation-only로 저장됨): {failed_npl[:3]}")


=== 순차 수집 시작: (거절-only 기본) 타겟 → 선행기술(특허/비특허) ===


Sequential collecting (targets→GT):   0%|          | 0/100 [00:00<?, ?it/s]

Failed to fetch KR101612025 B1: HTTP error / not found
Failed to fetch JP04190910 A: HTTP error / not found
Failed to fetch KR101576106 B1: HTTP error / not found
Failed to fetch KR101830056 B1: HTTP error / not found
Failed to fetch KR101932494 B1: HTTP error / not found
Failed to fetch KR101431429 B1: HTTP error / not found
Failed to fetch KR101002358 B1: HTTP error / not found
Failed to fetch KR101649173 B1: HTTP error / not found
Failed to fetch KR101783449 B1: HTTP error / not found
Failed to fetch KR101002358 B1: HTTP error / not found
Failed to fetch KR101649173 B1: HTTP error / not found
Failed to fetch KR101783449 B1: HTTP error / not found
Failed to fetch JP07244787 A: HTTP error / not found
Failed to fetch KR101803658 B1: HTTP error / not found
Failed to fetch US06594024 B1: HTTP error / not found
Failed to fetch KR101686919 B1: HTTP error / not found
Failed to fetch KR102032521 B1: HTTP error / not found
Failed to fetch KR101857624 B1: HTTP error / not found
Failed to fetch

Advanced encoding /KSCms-UHC-H not implemented yet
Advanced encoding /KSCms-UHC-H not implemented yet
Advanced encoding /KSCms-UHC-H not implemented yet
Advanced encoding /KSCms-UHC-H not implemented yet
Advanced encoding /KSCms-UHC-H not implemented yet
Advanced encoding /KSCms-UHC-H not implemented yet
Advanced encoding /KSCms-UHC-H not implemented yet
Advanced encoding /KSCms-UHC-H not implemented yet
Advanced encoding /KSCms-UHC-H not implemented yet
Advanced encoding /KSCms-UHC-H not implemented yet
Advanced encoding /KSCms-UHC-H not implemented yet
Advanced encoding /KSCms-UHC-H not implemented yet
Advanced encoding /KSCms-UHC-H not implemented yet
Advanced encoding /KSCms-UHC-H not implemented yet
Advanced encoding /KSCms-UHC-H not implemented yet
Advanced encoding /KSCms-UHC-H not implemented yet
Advanced encoding /KSCms-UHC-H not implemented yet
Advanced encoding /KSCms-UHC-H not implemented yet
Advanced encoding /KSCms-UHC-H not implemented yet
Advanced encoding /KSCms-UHC-H 

Failed to fetch KR102080712 B1: HTTP error / not found


Advanced encoding /KSCms-UHC-H not implemented yet
Advanced encoding /KSCms-UHC-H not implemented yet
Advanced encoding /KSCms-UHC-H not implemented yet
Advanced encoding /KSCms-UHC-H not implemented yet
Advanced encoding /KSCms-UHC-H not implemented yet
Advanced encoding /KSCms-UHC-H not implemented yet
Advanced encoding /KSCms-UHC-H not implemented yet
Advanced encoding /KSCms-UHC-H not implemented yet
Advanced encoding /KSCms-UHC-H not implemented yet
Advanced encoding /KSCms-UHC-H not implemented yet
Advanced encoding /KSCms-UHC-H not implemented yet
Advanced encoding /KSCms-UHC-H not implemented yet
Advanced encoding /KSCms-UHC-H not implemented yet
Advanced encoding /KSCms-UHC-H not implemented yet


Failed to fetch KR102050917 B1: HTTP error / not found
Failed to fetch KR101948236 B1: HTTP error / not found
Failed to fetch KR102050298 B1: HTTP error / not found
Failed to fetch KR101937434 B1: HTTP error / not found
Failed to fetch KR101527886 B1: HTTP error / not found
Failed to fetch KR101738899 B1: HTTP error / not found
Failed to fetch KR101387998 B1: HTTP error / not found
Failed to fetch KR102149671 B1: HTTP error / not found
Failed to fetch KR101995755 B1: HTTP error / not found
Failed to fetch KR102086659 B1: HTTP error / not found
Failed to fetch KR101995755 B1: HTTP error / not found
Failed to fetch JP60056853 A: HTTP error / not found
Failed to fetch KR101965605 B1: HTTP error / not found
Failed to fetch KR101508641 B1: HTTP error / not found
Failed to fetch KR102241357 B1: HTTP error / not found
Failed to fetch KR101989982 B1: HTTP error / not found
Failed to fetch KR102211049 B1: HTTP error / not found
Failed to fetch KR101734212 B1: HTTP error / not found
Failed to fe

In [7]:
# (호환) 선행기술 수집 셀
# - 현재 노트북은 '순차 수집(타겟→GT)'을 기본으로 하므로, 이 셀에서 추가 수집을 수행하지 않습니다.
# - prior_art_fulltext / failed_prior_arts / npl_fulltext 등은 이전 셀에서 채워집니다.

print("\n[INFO] 선행기술 수집은 이전 셀에서 entry 단위로 순차 수행되었습니다.")
print(f"- 선행기술(특허) 수집량: {len(globals().get('prior_art_fulltext', {}) or {})}")
print(f"- 선행기술(NPL) 수집량: {len(globals().get('npl_fulltext', {}) or {})}")


[INFO] 선행기술 수집은 이전 셀에서 entry 단위로 순차 수행되었습니다.
- 선행기술(특허) 수집량: 192
- 선행기술(NPL) 수집량: 19


In [8]:
# 메타데이터 생성 및 JSONL Export (파일 저장은 이미 완료됨)

print("\n=== 메타데이터 및 데이터셋 생성 (파일 저장은 수집 루프에서 완료됨) ===")

# 실패 사유 저장 (타겟)
fail_detail_path = OUT_DIR / "kipris_target_failures.json"
try:
    with open(fail_detail_path, "w", encoding="utf-8") as f:
        json.dump({
            "failed_targets": failed_targets,
            "failed_target_details": failed_target_details,
            "failed_target_reason_summary": globals().get("failed_target_reason_summary", {}),
        }, f, ensure_ascii=False, indent=2)
    print(f"✓ KIPRIS 타겟 실패 사유 저장: {fail_detail_path}")
except Exception as e:
    print(f"⚠️ 실패 사유 저장 실패: {e}")

# 메타데이터 저장 (UTF-8)
metadata = {
    "dataset_path": str(DATASET_PATH),
    "cohort": str(globals().get("COHORT", "unknown")),
    "sampling_method": str(globals().get("SAMPLING_METHOD", "unknown")),
    "sample_size": len(sample),
    "target_patents": {
        "total": len(target_app_nums),
        "collected": len(target_fulltext),
        "failed": len(failed_targets),
        "failed_reason_summary": globals().get("failed_target_reason_summary", {}),
    },
    "prior_arts": {
        "total_original_unique": len(set(prior_art_values)),
        "total_patent_like": len(prior_art_patent_doc_nums),
        "total_npl_like": len(prior_art_npl_values),
        "collected_patent": len(prior_art_fulltext),
        "failed_patent": len(failed_prior_arts),
        "collected_npl": len(npl_fulltext or {}),
        "failed_npl_web_extract": len(globals().get("failed_npl", []) or []),
        "korean": len(korean_prior_arts),
        "foreign": len(foreign_prior_arts),
    },
    "target_to_prior_arts": {
        entry["target_patent"]["application_number"]: (entry.get("ground_truth_prior_arts") or [])
        for entry in sample
    },
}

metadata_path = OUT_DIR / "fulltext_metadata.json"
with open(metadata_path, "w", encoding="utf-8") as f:
    json.dump(metadata, f, ensure_ascii=False, indent=2)

print(f"✓ 메타데이터 저장 (UTF-8): {metadata_path}")

# -------------------------
# 논문 평가용 JSONL export
# -------------------------
def extract_claim1(claims_text: str, *, max_chars: int = 1500) -> str:
    """Claim 1을 보수적으로 추출 (실패 시 claims 앞부분 fallback)."""
    if not claims_text:
        return ""
    t = str(claims_text).replace("\r", "\n")
    t = re.sub(r"\n{3,}", "\n\n", t).strip()
    patterns = [
        r"\[\s*청구항\s*1\s*\]\s*(.+?)(?=\[\s*청구항\s*2\s*\]|\Z)",
        r"청구항\s*1\s*[:\.-]?\s*(.+?)(?=청구항\s*2|\Z)",
        r"(?:^|\n)\s*1\s*[\.|\)]\s*(.+?)(?=(?:\n\s*2\s*[\.|\)]|\Z))",
    ]
    for pat in patterns:
        m = re.search(pat, t, flags=re.DOTALL)
        if m:
            c1 = re.sub(r"\s+", " ", m.group(1).strip()).strip()
            return c1[:max_chars]
    fallback = re.sub(r"\s+", " ", t).strip()
    return fallback[:max_chars]

def build_query_text(title: str, abstract: str, claim1: str) -> str:
    parts: List[str] = []
    if title:
        parts.append(f"Title: {title.strip()}")
    if abstract:
        parts.append(f"Abstract: {abstract.strip()}")
    if claim1:
        parts.append(f"Claim 1: {claim1.strip()}")
    return "\n\n".join(parts).strip()

def split_patent_vs_npl(values: List[str]) -> tuple[list[str], list[str]]:
    pats: list[str] = []
    npls: list[str] = []
    for v in values:
        if looks_like_patent_doc_number(v):
            pats.append(v)
        else:
            npls.append(v)
    return pats, npls

eval_records: list[dict] = []
missing_claim1 = 0
total_targets = 0

for entry in sample:
    tp = entry.get("target_patent", {}) or {}
    app_num = str(tp.get("application_number", "")).strip()
    if not app_num:
        continue
    total_targets += 1

    title = str(tp.get("title") or "").strip()
    abstract = str(tp.get("abstract") or "").strip()
    gt_all = [str(x) for x in (entry.get("ground_truth_prior_arts") or [])]
    gt_pat, gt_npl = split_patent_vs_npl(gt_all)

    claims_text = (target_fulltext.get(app_num) or {}).get("claims", "")
    claim1 = extract_claim1(claims_text)
    if not claim1:
        missing_claim1 += 1

    query_text = build_query_text(title, abstract, claim1)

    eval_records.append({
        "target_application_number": app_num,
        "query": {
            "title": title,
            "abstract": abstract,
            "claim1": claim1,
            "query_text": query_text,
        },
        "ground_truth": {
            "prior_arts_all": gt_all,
            "prior_arts_patent": gt_pat,
            "prior_arts_npl": gt_npl,
        },
        "corpus": {
            "targets_dir": str(target_dir),
            "prior_arts_dir": str(prior_art_dir),
            "npl_dir": str(npl_dir),
        },
    })

eval_path = OUT_DIR / "paper_eval_sample.jsonl"
with open(eval_path, "w", encoding="utf-8") as f:
    for r in eval_records:
        f.write(json.dumps(r, ensure_ascii=False) + "\n")

print(f"✓ 평가용 JSONL 저장: {eval_path} (targets={len(eval_records)})")
if total_targets:
    print(f"- Claim1 누락: {missing_claim1}/{total_targets} ({missing_claim1/total_targets*100:.1f}%)")

# -------------------------
# (추가) 원문 확보 실험용 데이터셋(JSONL) 생성: 샘플 100건 + 파일 경로 + 가용성
# -------------------------
fulltext_dataset_records: list[dict] = []
missing_target_txt = 0
missing_prior_txt = 0
missing_npl_txt = 0

def _npl_key(npl: str) -> str:
    import hashlib
    return hashlib.sha1(str(npl).encode("utf-8")).hexdigest()[:16]

for entry in sample:
    tp = entry.get("target_patent", {}) or {}
    app_num = str(tp.get("application_number", "")).strip()
    if not app_num:
        continue
    target_txt = target_dir / f"{re.sub(r'[^a-zA-Z0-9_-]', '_', app_num)}.txt"
    if not target_txt.exists():
        missing_target_txt += 1

    gt_all = [str(x) for x in (entry.get("ground_truth_prior_arts") or [])]
    gt_pat, gt_npl = split_patent_vs_npl(gt_all)

    prior_txt_paths: list[str] = []
    for doc in gt_pat:
        p = prior_art_dir / f"{re.sub(r'[^a-zA-Z0-9_-]', '_', str(doc))}.txt"
        if p.exists():
            prior_txt_paths.append(str(p))
        else:
            missing_prior_txt += 1

    npl_txt_paths: list[str] = []
    for npl in gt_npl:
        p = npl_dir / f"{_npl_key(npl)}.txt"
        if p.exists():
            npl_txt_paths.append(str(p))
        else:
            missing_npl_txt += 1

    fulltext_dataset_records.append({
        "target": {
            "application_number": app_num,
            "title": str(tp.get("title") or "").strip(),
            "abstract": str(tp.get("abstract") or "").strip(),
            "registration_number": (target_reg_num_by_app or {}).get(app_num, ""),
            "text_path": str(target_txt),
            "text_exists": target_txt.exists(),
            "kipris_operation": (target_fulltext.get(app_num) or {}).get("kipris_operation", ""),
        },
        "ground_truth": {
            "prior_arts_all": gt_all,
            "prior_arts_patent": gt_pat,
            "prior_arts_npl": gt_npl,
            "prior_art_text_paths": prior_txt_paths,
            "npl_text_paths": npl_txt_paths,
        },
        "dataset_source_path": str(DATASET_PATH),
    })

fulltext_dataset_path = OUT_DIR / f"paper_fulltext_dataset_sample{len(sample)}.jsonl"
with open(fulltext_dataset_path, "w", encoding="utf-8") as f:
    for r in fulltext_dataset_records:
        f.write(json.dumps(r, ensure_ascii=False) + "\n")

print(f"✓ 원문 확보 데이터셋(JSONL) 저장: {fulltext_dataset_path} (records={len(fulltext_dataset_records)})")
print(f"- 타겟 txt 누락: {missing_target_txt}")
print(f"- 선행기술 txt 누락(특허): {missing_prior_txt}")
print(f"- 선행기술 txt 누락(NPL): {missing_npl_txt}")


=== 메타데이터 및 데이터셋 생성 (파일 저장은 수집 루프에서 완료됨) ===
✓ KIPRIS 타겟 실패 사유 저장: /home/arkwith/Dev/paper_data/data/processed/fulltext/kipris_target_failures.json
✓ 메타데이터 저장 (UTF-8): /home/arkwith/Dev/paper_data/data/processed/fulltext/fulltext_metadata.json
✓ 평가용 JSONL 저장: /home/arkwith/Dev/paper_data/data/processed/fulltext/paper_eval_sample.jsonl (targets=100)
- Claim1 누락: 100/100 (100.0%)
✓ 원문 확보 데이터셋(JSONL) 저장: /home/arkwith/Dev/paper_data/data/processed/fulltext/paper_fulltext_dataset_sample100.jsonl (records=100)
- 타겟 txt 누락: 0
- 선행기술 txt 누락(특허): 66
- 선행기술 txt 누락(NPL): 0


In [9]:
# 수집 결과 요약

print("\n" + "=" * 60)
print("✅ 특허 원문 수집 완료")
print("=" * 60)

print(f"\n📊 타겟 특허 (한국)")
print(f"  - 대상: {len(target_app_nums)}건")
print(f"  - 성공: {len(target_fulltext)}건 ({len(target_fulltext)/len(target_app_nums)*100:.1f}%)")
print(f"  - 실패: {len(failed_targets)}건")
print(f"  - 소스: KIPRIS PDF API")

print(f"\n📊 선행기술")
print(f"  - 원본(중복 제거): {len(set(prior_art_values))}건")
print(f"  - 특허로 판별됨: {len(prior_art_patent_doc_nums)}건")
print(f"  - NPL로 판별됨(스킵): {len(prior_art_npl_values)}건")

if len(prior_art_patent_doc_nums) > 0:
    print(f"  - 수집 성공(특허만): {len(prior_art_fulltext)}건 ({len(prior_art_fulltext)/len(prior_art_patent_doc_nums)*100:.1f}%)")
else:
    print(f"  - 수집 성공(특허만): {len(prior_art_fulltext)}건")

print(f"  - 실패(특허만): {len(failed_prior_arts)}건")

# 소스별 분포
kipris_count = len([v for v in prior_art_fulltext.values() if v.get('source') == 'kipris_pdf'])
google_count = len([v for v in prior_art_fulltext.values() if v.get('source') == 'google_patents'])

print(f"\n📈 선행기술 소스 분포:")
print(f"  - 한국(KIPRIS PDF): {kipris_count}건")
print(f"  - Google Patents: {google_count}건")

print(f"\n💾 저장 위치")
print(f"  - 타겟 텍스트: {target_dir}")
print(f"  - 선행기술 텍스트: {prior_art_dir}")
print(f"  - PDF 캐시: {PDF_DIR}")
print(f"  - 메타데이터: {metadata_path}")

print(f"\n✅ 다음 단계: RAG 벡터스토어 구축")
print(f"  → 텍스트 청킹 및 임베딩 생성")
print(f"  → DB 저장 및 키워드 색인")

# 인코딩 검증 (샘플 파일)
sample_file = list(prior_art_dir.glob("*.txt"))
if sample_file:
    sample_file = sample_file[0]
    with open(sample_file, "r", encoding="utf-8") as f:
        content = f.read()[:300]

    print(f"\n🔍 UTF-8 인코딩 검증 (샘플 파일):")
    print(f"  파일: {sample_file.name}")
    print(f"  내용 미리보기: {content[:120]}...")

    has_korean = bool(re.search(r"[가-힣]", content))
    has_chinese = bool(re.search(r"[\u4e00-\u9fff]", content))
    has_japanese = bool(re.search(r"[\u3040-\u309f\u30a0-\u30ff]", content))

    if has_korean or has_chinese or has_japanese:
        print(f"  ✓ 다국어 문자 정상 감지됨 (한국어: {has_korean}, 중국어: {has_chinese}, 일본어: {has_japanese})")
    else:
        print("  ⚠️ 샘플에서 CJK 문자가 감지되지 않았습니다. (문서가 영어일 수 있음)")


✅ 특허 원문 수집 완료

📊 타겟 특허 (한국)
  - 대상: 100건
  - 성공: 100건 (100.0%)
  - 실패: 0건
  - 소스: KIPRIS PDF API

📊 선행기술
  - 원본(중복 제거): 273건
  - 특허로 판별됨: 254건
  - NPL로 판별됨(스킵): 19건
  - 수집 성공(특허만): 192건 (75.6%)
  - 실패(특허만): 66건

📈 선행기술 소스 분포:
  - 한국(KIPRIS PDF): 115건
  - Google Patents: 77건

💾 저장 위치
  - 타겟 텍스트: /home/arkwith/Dev/paper_data/data/processed/fulltext/targets
  - 선행기술 텍스트: /home/arkwith/Dev/paper_data/data/processed/fulltext/prior_arts
  - PDF 캐시: /home/arkwith/Dev/paper_data/data/processed/fulltext_pdfs
  - 메타데이터: /home/arkwith/Dev/paper_data/data/processed/fulltext/fulltext_metadata.json

✅ 다음 단계: RAG 벡터스토어 구축
  → 텍스트 청킹 및 임베딩 생성
  → DB 저장 및 키워드 색인

🔍 UTF-8 인코딩 검증 (샘플 파일):
  파일: KR1020190097397_A.txt
  내용 미리보기: Document Number: KR1020190097397 A
Source: kipris_pdf
KIPRIS Operation: getPubFullTextInfoSearch
Title: 
===============...
  ✓ 다국어 문자 정상 감지됨 (한국어: True, 중국어: False, 일본어: False)
